#### I am crating this notebook since I realised that I should consolidate all the preprocessing of the data before training

In [1]:
import pandas as pd
import numpy as np
import os
from ast import literal_eval

In [2]:
df = pd.read_csv('/Users/louishagenbucher/Documents/GitHub/mtg_nlp_price_prediction/data/cards_data.csv')
df.sample(5)

,id,name,cmc,mana_cost,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,textless,rarity,reserved,released_at,booster,usd
7817,a93c6ba8-666b-4c05-8137-8ffa1d5d928b,Flame Blitz,1.0,{R},['R'],False,['Enchantment'],['none'],"At the beginning of your end step, Flame Blitz...",['Cycling'],non-creature,non-creature,False,uncommon,False,2021-06-18,True,0.04
3232,4d485efd-667f-4561-b747-7605bf1ebc09,Carrion Grub,4.0,{3}{B},['B'],False,['Creature'],['Insect'],"Carrion Grub gets +X/+0, where X is the greate...",['Mill'],0,5,False,common,False,2023-08-04,True,0.03
870,7c247fd6-09f3-40cd-b3fb-53b6526e1dcf,Arcane Lighthouse,0.0,{0},['No Colors'],False,['Land'],['none'],"{T}: Add {C}.\n{1}, {T}: Until end of turn, cr...",['no keywords'],non-creature,non-creature,False,uncommon,False,2023-08-04,False,3.99
3663,312505d7-362e-43cf-bd23-28c248a8b7e1,Chill to the Bone,4.0,{3}{B},['B'],False,['Instant'],['none'],Destroy target nonsnow creature.,['no keywords'],non-creature,non-creature,False,common,False,2006-07-21,True,0.11
12011,e6246cf3-76bd-476b-9cd9-789b6ad48887,Kheru Mind-Eater,3.0,{2}{B},['B'],False,['Creature'],['Vampire'],Menace\nWhenever Kheru Mind-Eater deals combat...,['Menace'],1,3,False,rare,False,2017-08-25,False,0.84


In [3]:
df.drop(columns=['id', 'mana_cost', 'textless'], inplace=True)

In [4]:
df['sub_types'].fillna("['none']", inplace=True)

In [5]:
df['types'] = df['types'].apply(literal_eval)
df['sub_types'] = df['sub_types'].apply(literal_eval)
df['keywords'] = df['keywords'].apply(literal_eval)

In [6]:
df.drop(df[df['colors']=='[nan]'].index, inplace=True)
df.reset_index(drop=True)
df.head()

,name,cmc,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,rarity,reserved,released_at,booster,usd
0,+2 Mace,2.0,['W'],False,[Artifact],[Equipment],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,[Equip],non-creature,non-creature,common,False,2021-07-23,True,0.02
1,Aarakocra Sneak,4.0,['U'],False,[Creature],"[Bird, Rogue]",Flying\nWhen Aarakocra Sneak enters the battle...,[Flying],1,4,common,False,2022-06-10,True,0.06
2,Abaddon the Despoiler,5.0,"['B', 'R', 'U']",True,[Creature],"[Astartes, Warrior]",Trample\nMark of Chaos Ascendant — During your...,"[Mark of Chaos Ascendant, Trample]",5,5,mythic,False,2022-10-07,False,2.81
3,Abandoned Outpost,0.0,['No Colors'],False,[Land],[none],Abandoned Outpost enters the battlefield tappe...,[no keywords],non-creature,non-creature,common,False,2001-10-01,True,0.14
4,Abandoned Sarcophagus,3.0,['C'],False,[Artifact],[none],You may cast spells that have a cycling abilit...,[no keywords],non-creature,non-creature,rare,False,2020-04-17,False,0.09


In [7]:
df['colors'].replace("['No Colors']", "['N']", inplace=True)
df['colors']=df['colors'].apply(literal_eval)
df.reset_index(drop=True, inplace=True)
df['colors'].head()

0          [W]
1          [U]
2    [B, R, U]
3          [N]
4          [C]
Name: colors, dtype: object

In [8]:
df['power'] = df['power'].replace('non-creature', 0)
df['toughness'] = df['toughness'].replace('non-creature', 0)

In [9]:
df.drop(df[df['types'].apply(lambda x: '//' in x)].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [10]:
null_texts_ind = df[df['name'].str.contains('//')][['name','oracle_text']].index
df.drop(null_texts_ind, inplace=True)
df.reset_index(drop=True).head()

,name,cmc,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,rarity,reserved,released_at,booster,usd
0,+2 Mace,2.0,[W],False,[Artifact],[Equipment],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,[Equip],0,0,common,False,2021-07-23,True,0.02
1,Aarakocra Sneak,4.0,[U],False,[Creature],"[Bird, Rogue]",Flying\nWhen Aarakocra Sneak enters the battle...,[Flying],1,4,common,False,2022-06-10,True,0.06
2,Abaddon the Despoiler,5.0,"[B, R, U]",True,[Creature],"[Astartes, Warrior]",Trample\nMark of Chaos Ascendant — During your...,"[Mark of Chaos Ascendant, Trample]",5,5,mythic,False,2022-10-07,False,2.81
3,Abandoned Outpost,0.0,[N],False,[Land],[none],Abandoned Outpost enters the battlefield tappe...,[no keywords],0,0,common,False,2001-10-01,True,0.14
4,Abandoned Sarcophagus,3.0,[C],False,[Artifact],[none],You may cast spells that have a cycling abilit...,[no keywords],0,0,rare,False,2020-04-17,False,0.09


In [11]:
df['oracle_text'].fillna('notext', inplace=True)
df.drop(columns=['name'], inplace=True)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25424 entries, 0 to 25544
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   cmc          25424 non-null  float64
 1   colors       25424 non-null  object 
 2   legendary    25424 non-null  bool   
 3   types        25424 non-null  object 
 4   sub_types    25424 non-null  object 
 5   oracle_text  25424 non-null  object 
 6   keywords     25424 non-null  object 
 7   power        25424 non-null  object 
 8   toughness    25424 non-null  object 
 9   rarity       25424 non-null  object 
 10  reserved     25424 non-null  bool   
 11  released_at  25424 non-null  object 
 12  booster      25424 non-null  bool   
 13  usd          25368 non-null  float64
dtypes: bool(3), float64(2), object(9)
memory usage: 2.4+ MB


In [13]:
directory_path = '/Users/louishagenbucher/Documents/GitHub/mtg_nlp_price_prediction/data'
df.to_csv(os.path.join(directory_path, 'clean_cards.csv'), index=False)